In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage
import pandas


cred = credentials.Certificate("admin-sdk.json")
firebase_admin.initialize_app(cred,{
    'databaseURL': 'https://iot-solar-database-default-rtdb.firebaseio.com/',
    'storageBucket':'iot-solar-database.appspot.com'
})

bucket = storage.bucket()

In [83]:
df_g = pandas.read_csv('dataset-solar\Plant_1_Generation_Data.csv')
df_w = pandas.read_csv('dataset-solar\Plant_1_Weather_Sensor_Data.csv')

In [84]:
df_g['DATE'],df_g['TIME']=df_g['DATE_TIME'].str.split(' ',1).str
df_w['DATE'],df_w['TIME']=df_w['DATE_TIME'].str.split(' ',1).str
del df_g['DATE_TIME']
del df_w['DATE_TIME']
df_g = df_g[ ['TIME'] + [ col for col in df_g.columns if col != 'TIME' ] ]
df_w = df_w[ ['TIME'] + [ col for col in df_w.columns if col != 'TIME' ] ]
df_g = df_g[ ['DATE'] + [ col for col in df_g.columns if col != 'DATE' ] ]
df_w = df_w[ ['DATE'] + [ col for col in df_w.columns if col != 'DATE' ] ]

In [87]:
del df_g['DAILY_YIELD']
del df_g['TOTAL_YIELD']
df_g
df_w

,DATE,TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,15/05/2020,0:00,25.184316,22.857507,0.0
1,15/05/2020,0:15,25.084589,22.761668,0.0
2,15/05/2020,0:30,24.935753,22.592306,0.0
3,15/05/2020,0:45,24.846130,22.360852,0.0
4,15/05/2020,1:00,24.621525,22.165423,0.0
...,...,...,...,...,...
3177,17/06/2020,22:45,22.150570,21.480377,0.0
3178,17/06/2020,23:00,22.129816,21.389024,0.0
3179,17/06/2020,23:15,22.008275,20.709211,0.0
3180,17/06/2020,23:30,21.969495,20.734963,0.0


In [88]:
df_g = df_g.groupby(['DATE','TIME'], as_index=False).mean()
df_g

,DATE,TIME,DC_POWER,AC_POWER
0,01/06/2020,0:00,0.000000,0.000000
1,01/06/2020,0:15,0.000000,0.000000
2,01/06/2020,0:30,0.000000,0.000000
3,01/06/2020,0:45,0.000000,0.000000
4,01/06/2020,10:00,7754.196429,758.716883
...,...,...,...,...
3153,31/05/2020,8:45,6915.861201,677.093344
3154,31/05/2020,9:00,7026.840097,688.065097
3155,31/05/2020,9:15,5312.683442,520.857630
3156,31/05/2020,9:30,8345.980519,816.289773


In [89]:
import pandas as pd
df_final = pd.merge(df_w, df_g_t, on=['DATE','TIME'])
del pd
df_final

,DATE,TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,DC_POWER,AC_POWER
0,15/05/2020,0:00,25.184316,22.857507,0.0,0.0,0.0
1,15/05/2020,0:15,25.084589,22.761668,0.0,0.0,0.0
2,15/05/2020,0:30,24.935753,22.592306,0.0,0.0,0.0
3,15/05/2020,0:45,24.846130,22.360852,0.0,0.0,0.0
4,15/05/2020,1:00,24.621525,22.165423,0.0,0.0,0.0
...,...,...,...,...,...,...,...
3152,17/06/2020,22:45,22.150570,21.480377,0.0,0.0,0.0
3153,17/06/2020,23:00,22.129816,21.389024,0.0,0.0,0.0
3154,17/06/2020,23:15,22.008275,20.709211,0.0,0.0,0.0
3155,17/06/2020,23:30,21.969495,20.734963,0.0,0.0,0.0


In [90]:
json_final = df_final.to_dict('index')

In [91]:
ref = db.reference('Plant_Generation+Sensor_Data/')

In [92]:
ref.set(json_final)

In [93]:
df_final.to_csv(r'dataset-solar\Plant_1_generation+sensor.csv', index = False)